In [1]:
import arcpy
import psycopg2
import requests
import json
import os
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

In [2]:
# Set environment
arcpy.env.workspace = r'E:\ArcGIS_2\Lab1'
wksp = arcpy.env.workspace

In [3]:
# Create List of Points
input_pts= [arcpy.Point(2, 5), arcpy.Point(6, 8), arcpy.Point(10, 20), arcpy.Point(20, 1), arcpy.Point(2, 5)]
# Convert to Polygon
poly = arcpy.Polygon(arcpy.Array(input_pts))

In [9]:
# Connect to postgresql database
# Create unique database for this lab
# Create cursor for the table
conection = psycopg2.connect(host = '34.27.219.64',
                              port = '5432',
                              database = 'lab1',
                              user = 'postgres',
                              password = 'student',
                             )
cursor = conection.cursor()

# Create and populate table in the database
# Only create if one does not exist
# Serial key will keep track of the features, create column with geomtery of features
# Use default spatial reference 4326,
cursor.execute("CREATE TABLE IF NOT EXISTS poly (id SERIAL PRIMARY KEY, geom GEOMETRY)")
cursor.execute("INSERT INTO poly (geom) VALUES (ST_GeomFromText('{}', 4326))".format(poly.WKT))
# Add the information to the table
conection.commit()
conection.close()

In [11]:
# Retrieve polygon from VM API
api = r'http://34.29.175.222:5000/geojson_polygon'
dictionary = json.loads(requests.get(api).text)

# Remove one pair of brackets  
sub = dictionary['geojson'].replace("[[[[", '[[[').replace(']]]]', ']]]')

# Convert string to JSON dictionary
dictionary_2 = json.loads(sub)

In [6]:
# Create GeoJSON dictionary
geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": dictionary_2['coordinates'],
        "type": "Polygon"
      }
    }
  ]
}

# Create file with the GeoJSON dictionary
path = os.path.join(wksp, 'geojson.json')
with open (path, 'w') as json_file:
    json.dump(geojson, json_file)

In [8]:
# Sign in to ArcGIS Online
username = ''
password = ''

gis = GIS('https://www.arcgis.com', username, password)

# Publish layer onto ArcGIS Online
item = gis.content.add({'type': 'GeoJson'}, 'geojson.json')
feature_layer_item = item.publish()
feature_layer_url = feature_layer_item.url